In [1]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import pickle
import glob
import pandas as pd

#data augmentation
#import local version of the library imgaug
sys.path.append('C:\\Users\\camil\\Desktop\\animals_code\\imgaug')
import imgaug as ia
from imgaug import augmenters as iaa

#unbalanced dataset
from sklearn.utils import class_weight

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")
path_image = os.path.join('datasets','KBF','ANNOTATED_IMAGES')

#import local version of the library of Mask RCNN
sys.path.append('C:\\Users\\camil\\Desktop\\animals_code\\Mask_RCNN')
import tensorflow as tf
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join('C:\\Users\\camil','Desktop','animals_code','Mask_RCNN', "mask_rcnn_coco_resnet101.h5")
IMAGENET_WEIGHTS_PATH = os.path.join('C:\\Users\\camil','Desktop','animals_code','Mask_RCNN', "resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")
# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
LOGS_DIRECTORY = os.path.join('C:\\Users\\camil\Desktop\\animals_code\\PhD-AnimalWelfare','KBF_CNN','LOGS')

In [2]:
#GPU issue (595): for fast update to monitoring it in real time. If it is utilized some, but not all the time, it means that
#it waits for CPU to finish and halt, it means that your load_mask or load_image function is slow.
#os.environ['CUDA_VISIBLE_DEVICES'] = '0' TODO: check what this is 

In [3]:
#on command prompt: conda install cudatoolkit

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [5]:
import utils_data_class_and_config

In [6]:
#images must be all in the path_image folder for the algo to use them. We will simply provide the algo the name of the image 
#and the path

# Configurations

In [7]:
config = utils_data_class_and_config.DataConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        8
DETECTION_MIN_CONFIDENCE       0.65
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                17
IMAGE_MIN_DIM                  170
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.0005
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE              

# Data augmentation

In [8]:
aug = iaa.Sometimes(0.85,[
        
    #Sharpen or emboss an image: 'see more the pixels' & contrast
    iaa.SomeOf(0.2,[iaa.Sharpen(alpha=(0.5,1), lightness=(1)),
                  iaa.Emboss(alpha=(0.5,1), strength=(1.0,1.7)),
                   iaa.ContrastNormalization((0.5,1.5))]),
    
    #dangerous with our data
    #Add r values between -30 and 30 to images. In 50% of all images the values differ per channel (3 sampled value). 
    #In the other 50% of all images the value is the same for all channels: (change color constancy)
    #iaa.Add((-20, 20), per_channel=0.5),
    #Increase/decrease S (saturation i.e. how colorful it is)
    #iaa.WithColorspace(to_colorspace="HSV", from_colorspace="RGB", 
    #                         children=iaa.WithChannels(1, iaa.Multiply((0.65,1.35)))),
    #not good as it can be part of caracteristic for species
    #scale image independantly in x and y axis 
    #iaa.Affine(scale={"x": (0.75, 1.25), "y": (0.75, 1.25)}),
    #distortion
    #iaa.PiecewiseAffine(scale=(0.01, 0.03)),
    
    #darkness/brightness choose one of both technique
    iaa.Sometimes(0.2,iaa.SomeOf(1,[iaa.Multiply((0.75, 1.25)),
                                    #Increase/decrease V (value i.e. how bright/dark) 
                                    iaa.WithColorspace(to_colorspace="HSV", from_colorspace="RGB", 
                                                       children=iaa.WithChannels(2, iaa.Multiply((0.75, 1.25))))])),
    
    #flip horizontaly and vertically to make as if the fish was swimimg from both direction and from upside down
    iaa.Fliplr(0.3),
    iaa.Flipud(0.15),
    iaa.Sometimes(0.3,iaa.Affine(rotate=(-15, 15))),
    #iaa.Sometimes(0.5,iaa.SomeOf(1, [iaa.Affine(rotate=15),iaa.Affine(rotate=30),iaa.Affine(rotate=45),
    #                                 iaa.Affine(rotate=5),iaa.Affine(rotate=10),iaa.Affine(rotate=25),
    #                                 iaa.Affine(rotate=60),iaa.Affine(rotate=75),iaa.Affine(rotate=0)]))
    
    #weather: Clouds, Fog, Snowflakes & noise #iaa.Fog(),iaa.Snowflakes(density=(0.005, 0.025),flake_size=(0.2, 1.0))
    #noise: 
    iaa.Sometimes(0.2,iaa.SomeOf(1,[iaa.AdditiveGaussianNoise(scale=0.007*255), 
                                    iaa.AverageBlur(k=(2, 5))])),
    #AttributeError: 'CloudLayer' object has no attribute 'density_min'
    #iaa.Sometimes(0.2,[iaa.SomeOf(1,[iaa.Clouds(),
    #                                 iaa.Snowflakes(density=(0.005, 0.025),flake_size=(0.2, 1.0))])]),
    
    #resize (not bigger as not all fish can be bigger, but smaller in theory yes)
    iaa.Sometimes(0.5,iaa.Resize((0.5, 1.0)))
])

# Create the model

In [9]:
#verify if tensorflow with GPU is well installed : it works!
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10057796542673430702
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14448868414071970151
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 17878557664136668624
physical_device_desc: "device: XLA_GPU device"
]


#Device to load the neural network on.
#Useful if you're training a model on the same 
#machine, in which case use CPU and leave the
#GPU for training.
#DEVICE = "/GPU:0"  # /cpu:0 or /gpu:0
with tf.device("/GPU:0"):
    model = modellib.MaskRCNN(mode='training', model_dir=LOGS_DIRECTORY, config=config)
#hence can not change config and using same model, its initiate with this config now

In [10]:
model = modellib.MaskRCNN(mode='training', model_dir=LOGS_DIRECTORY, config=config)

# Train

In [11]:
def train(model, firstepoch=0, second=25, third=5, fourth=3):
    
    path_ = path_image

    ########################################################################################   
    #print('Fine tune Resnet stage 4 and up with algue to be able to distinguish between both')
    
    #Finetune layers from ResNet stage 4 and up
    #training dataset.
    '''dataset_train = utils_data_class_and_config.VGG_Dataset()
    dataset_train.load_vgg(path_, "train")
    dataset_train.prepare()
    
    #validation dataset
    dataset_val = utils_data_class_and_config.VGG_Dataset()
    dataset_val.load_vgg(path_, "val")
    dataset_val.prepare()
    
    #training
    model.train(dataset_train, dataset_val,
                learning_rate = config.LEARNING_RATE,
                epochs = firstepoch,
                layers = 'all', augmentation=aug)'''
    
    ########################################################################################
    #print('Fine tune all layers without algue')
    
    #training dataset.
    dataset_train = utils_data_class_and_config.VGG_Dataset()
    dataset_train.load_vgg(path_, "train")
    dataset_train.prepare()
    
    #validation dataset
    dataset_val = utils_data_class_and_config.VGG_Dataset()
    dataset_val.load_vgg(path_, "val")
    dataset_val.prepare()
    
    print('start training')
    #training
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=firstepoch+second,
                layers='all') #, augmentation=aug
    
    '''model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/10,
                epochs=firstepoch+second+third,
                layers='all', augmentation=aug)
    
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/100,
                epochs=firstepoch+second+third+fourth,
                layers='all')'''

In [12]:
#Exclude the last layers because they require a number of classes matching
WEIGHTS_PATH = IMAGENET_WEIGHTS_PATH
WEIGHTS_PATH = COCO_WEIGHTS_PATH
model.load_weights(WEIGHTS_PATH, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

#my ones
#WEIGHTS_PATH = os.path.join(LOGS_DIRECTORY, '?', 'mask_rcnn_dog_flickr_0400.h5')
#model.load_weights(WEIGHTS_PATH, by_name=True)

In [13]:
#train or evaluate
train(model, 0, 30)

start training

Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\camil\Desktop\animals_code\PhD-AnimalWelfare\KBF_CNN\LOGS\kbf_flickr20210806T1046\mask_rcnn_kbf_flickr_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a

C:\Users\camil\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:431: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


15/15 [==============================] - ETA: 0s - batch: 7.0000 - size: 2.0000 - loss: 3.2621 - rpn_class_loss: 0.0998 - rpn_bbox_loss: 2.0566 - mrcnn_class_loss: 0.5273 - mrcnn_bbox_loss: 0.3394 - mrcnn_mask_loss: 0.2390 WARNING:tensorflow:From C:\Users\camil\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training_v1.py:2048: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
15/15 [==============================] - 184s 12s/step - batch: 7.0000 - size: 2.0000 - loss: 3.2621 - rpn_class_loss: 0.0998 - rpn_bbox_loss: 2.0566 - mrcnn_class_loss: 0.5273 - mrcnn_bbox_loss: 0.3394 - mrcnn_mask_loss: 0.2390 - val_loss: 1.4434 - val_rpn_class_loss: 0.0564 - val_rpn_bbox_loss: 1.3870 - val_mrcnn_class_loss: 9.2089e-06 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 2/30


Epoch 20/30
15/15 [==============================] - 177s 12s/step - batch: 7.0000 - size: 2.0000 - loss: 1.5493 - rpn_class_loss: 0.0384 - rpn_bbox_loss: 1.1970 - mrcnn_class_loss: 0.0033 - mrcnn_bbox_loss: 0.2039 - mrcnn_mask_loss: 0.1068 - val_loss: 1.8648 - val_rpn_class_loss: 0.0395 - val_rpn_bbox_loss: 1.3868 - val_mrcnn_class_loss: 0.0048 - val_mrcnn_bbox_loss: 0.2992 - val_mrcnn_mask_loss: 0.1345
Epoch 21/30
15/15 [==============================] - 183s 12s/step - batch: 7.0000 - size: 2.0000 - loss: 1.5398 - rpn_class_loss: 0.0345 - rpn_bbox_loss: 0.9747 - mrcnn_class_loss: 0.0063 - mrcnn_bbox_loss: 0.3245 - mrcnn_mask_loss: 0.1997 - val_loss: 2.1079 - val_rpn_class_loss: 0.0381 - val_rpn_bbox_loss: 1.2182 - val_mrcnn_class_loss: 0.0072 - val_mrcnn_bbox_loss: 0.3607 - val_mrcnn_mask_loss: 0.4837
Epoch 22/30
15/15 [==============================] - 181s 12s/step - batch: 7.0000 - size: 2.0000 - loss: 0.9816 - rpn_class_loss: 0.0301 - rpn_bbox_loss: 0.7449 - mrcnn_class_loss: 0.

In [14]:
#Once TensorBoard is running, navigate your web browser to localhost:6006 to view the TensorBoard.
#tensorboard --logdir=C:\Users\camil\Desktop\animals_code\PhD-AnimalWelfare\KBF_CNN\LOGS

In [15]:
#TODO: remove some class!!!
#try without dog.. oke?
# si non try with all
#si non try with only dog then with all 